## Factor graph example

In [9]:
import symforce
symforce.set_epsilon_to_symbol()
import symforce.symbolic as sf
import numpy as np
from symforce.values import Values
from symforce import typing as T

AlreadyUsedEpsilon: Cannot set return value of epsilon after it has already been called.

In [2]:
# 3 satellites, 3 poses
num_poses = 3
num_satellites = 3

In [3]:
# Ground truth positions
x_pos = np.array([[3, 1], [7, 4], [11, 3]])
s_pos = np.array([[-2, 7], [13, 9], [7, -6]])

# Ground truth distances
ranges = np.zeros((num_satellites, num_poses))
for i in range(num_satellites):
    for j in range(num_poses):
        ranges[i, j] = np.linalg.norm(x_pos[j, :] - s_pos[i, :])

In [4]:
# Ground truth headings
x1_heading = np.arctan2(4, 3)
x2_heading = np.arctan2(4, -1)
x3_heading = 0

In [5]:
# Store in values
initial_values = Values(
    poses=[sf.Pose2.identity()] * num_poses,
    satellites=[sf.V2(-2, 7), sf.V2(13, 9), sf.V2(7, -6)],
    odometry=[sf.Pose2(R = sf.Rot2.from_angle(x2_heading-x1_heading), t = sf.V2(4, 3)), 
               sf.Pose2(R = sf.Rot2.from_angle(x3_heading-x2_heading), t = sf.V2(4, -1))],
    ranges=ranges,
    epsilon=sf.numeric_epsilon,
)

In [11]:
sf.Rot3()

<Rot3 <Q xyzw=[0, 0, 0, 1]>>

In [13]:
sf.Pose3(R = sf.Rot3(), t = sf.V3(4, 3, 2))

<Pose3 R=<Rot3 <Q xyzw=[0, 0, 0, 1]>>, t=(4, 3, 2)>

In [10]:
p1 = sf.Pose2(R = sf.Rot2.from_angle(2.0), t = sf.V2(4, 3))
p2 = sf.Pose2(R = sf.Rot2.from_angle(1.0), t = sf.V2(2, 4))

In [11]:
p2.compose(p1.inverse())

<Pose2 R=<Rot2 <C real=0.54030230586814, imag=-0.841470984807897>>, t=(-2.68562217789625, 5.74497702162717)>

In [22]:
p2

<Pose2 R=<Rot2 <C real=0.54030230586814, imag=0.841470984807897>>, t=(2, 4)>

In [25]:
display(T.cast(sf.V3, p2))

<Pose2 R=<Rot2 <C real=0.54030230586814, imag=0.841470984807897>>, t=(2, 4)>

In [6]:
def range_residual(
    pose: sf.Pose2, satellite: sf.V2, range: sf.Scalar, epsilon: sf.Scalar
) -> sf.V1:
    return sf.V1((satellite - pose.t).norm(epsilon=epsilon) - range)

In [7]:
def odometry_residual(
    pose_a: sf.Pose2, pose_b: sf.Pose2, odom: sf.Pose2, epsilon: sf.Scalar
) -> sf.V3:
    pose_diff = pose_b.compose(pose_a.inverse())
    return T.cast(sf.V3, pose_diff.compose(odom.inverse()))

In [8]:
from symforce.opt.factor import Factor

factors = []

# Range factors
for i in range(num_poses):
    for j in range(num_satellites):
        factors.append(Factor(
            residual=range_residual,
            keys=[f"poses[{i}]", f"satellites[{j}]", f"ranges[{i}][{j}]", "epsilon"],
        ))

# Odometry factors
for i in range(num_poses - 1):
    factors.append(Factor(
        residual=odometry_residual,
        keys=[f"poses[{i}]", f"poses[{i + 1}]", f"odometry[{i}]", "epsilon"],
    ))

codegen.__init__():123 WARNING -- 
    Generating code with epsilon set to 0 - This is dangerous!  You may get NaNs, Infs,
    or numerically unstable results from calling generated functions near singularities.

    In order to safely generate code, you should set epsilon to either a symbol
    (recommended) or a small numerical value like `sf.numeric_epsilon`.  You should do
    this before importing any other code from symforce, e.g. with

        import symforce
        symforce.set_epsilon_to_symbol()

    or

        import symforce
        symforce.set_epsilon_to_number()

    For more information on use of epsilon to prevent singularities, take a look at the
    Epsilon Tutorial: https://symforce.org/tutorials/epsilon_tutorial.html

codegen.__init__():123 WARNING -- 
    Generating code with epsilon set to 0 - This is dangerous!  You may get NaNs, Infs,
    or numerically unstable results from calling generated functions near singularities.

    In order to safely generate code

In [27]:
from symforce.opt.optimizer import Optimizer

optimizer = Optimizer(
    factors=factors,
    optimized_keys=[f"poses[{i}]" for i in range(num_poses)],
    # So that we save more information about each iteration, to visualize later:
    debug_stats=True,
)

ValueError: The output of a factor must be a column vector representing the residual (of shape Nx1).  For factor "odometry_residual", got an object of type <class 'symforce.geo.pose2.Pose2'> instead